# Processamento Digital de Imagens

1001527 - Turma A

Prof. Cesar Henrique Comin

## Sistema para desenhar no ar com um objeto colorido

### Equipe
Igor Teixeira Machado RA: 769708

Rafael Vinícius Passador RA: 790036

* pip install opencv-python
* pip install numpy
* python -m pip install scipy

In [57]:
import cv2
import numpy as np
import scipy.ndimage as ndi
import matplotlib.pyplot as plt

In [58]:
def detect_object(frame):

    # Convertendo para HSV.
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Escolhendo limites para a detecção da cor do objeto vermelho.
    #lower_bound = np.array([10, 100, 20])
    #upper_bound = np.array([25, 255, 255])

    # Escolhendo limites para a detecção da cor do objeto azul.
    lower_bound = np.array([90, 50, 70])
    upper_bound = np.array([128, 255, 255])

    # Aplicando os limites.
    mask = cv2.inRange(hsv, lower_bound, upper_bound)

    # Definindo o tamanho do kernel.
    kernel = np.ones((5, 5), np.uint8)

    mask = cv2.dilate(mask, kernel, iterations=1)

    return mask

def mass_center(mask, img):

    contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    matrix = cv2.moments(contours[0])
    center = (int(matrix['m10'] / matrix['m00']), int(matrix['m01'] / matrix['m00']))

    return center

def draw_circle(image, coord, radius=5, color=(0, 0, 255), thickness=-1):

    # cor: BGR
    image = cv2.circle(image, coord, radius, color, thickness)

    return image

In [59]:
# Abre dispositivo de captura. O parâmetro 0 faz com que
# a primeira câmera encontrada seja utilizada
vcap = cv2.VideoCapture(0)

while True:
    ret, curr_frame = vcap.read()
    if ret:
        # Redimensionamos a imagem para diminuir o custo computacional
        # necessário para processar o frame
        resized_frame = cv2.resize(curr_frame, (640, 360))
        #print(resized_frame.shape)
        #processed_frame = process_frame(resized_frame)
        paint_object = detect_object(resized_frame)
        
        object_mass_center = mass_center(paint_object, resized_frame)
        processed_frame = draw_circle(resized_frame, object_mass_center)
               
        cv2.imshow('frame', processed_frame)
    else:
        print("Frame not available")

    # Esperar 33 milisegundos significa mostrar 30 imagens por segundo (1/0.033).
    # Portanto, o FPS (frames per second) do vídeo mostrado será aproximadamente 30.
    # O valor não é exatamente 30 porque um tempo é gasto no processamento do frame
    key = cv2.waitKey(33) & 0xff

    if key == 27:
        break

vcap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1